In [1]:
import plotly.graph_objects as go
import plotly.express as px
import datetime as dt
from plotly.subplots import make_subplots
import datetime as dt


import collections
import numpy as np
import pandas as pd
import os
import math
import json

In [15]:
# path = "../data/ontario/cases.jsonl"
# with open(path, 'r') as outfile:
#     all_cases = [ json.loads(c) for c in outfile]
# df =  pd.DataFrame.from_dict(all_cases)

path = "../data/ontario/updates.jsonl"
with open(path, 'r') as outfile:
    updates = [ json.loads(c) for c in outfile]

updates = sorted(updates, key=lambda k: k['date']) 

In [16]:
# cities = list(df.city.unique())

# infections = [[case["date"][:10] for case in all_cases if case["city"]== city and case["date"][:10] > "2020-02-01"] for city in cities ]


dates = { t['date'][:10]:i for i, t in enumerate(updates)}

unique_dates_ix = list(dates.values())
dates = list(dates.keys())

# Total Cases

In [17]:


negative = [t['negative'] for i,t in enumerate(updates) if i in unique_dates_ix]
under = [t['pending'] for i,t in enumerate(updates)  if i in unique_dates_ix]
positive = [t['positive'] for i,t in enumerate(updates)  if i in unique_dates_ix]
resolved = [t['resolved'] for i,t in enumerate(updates) if i in unique_dates_ix]
dead = [t['deceased'] for i,t in enumerate(updates)  if i in unique_dates_ix]
total = [t['total'] for i,t in enumerate(updates)  if i in unique_dates_ix]
hospitalization = [t['hospitalizations'] for i,t in enumerate(updates)  if i in unique_dates_ix and 'hospitalizations' in t]
icu = [t['ICU'] for i,t in enumerate(updates)  if i in unique_dates_ix and 'ICU' in t]
ventilator = [t['ventilator'] for i,t in enumerate(updates)  if i in unique_dates_ix and 'ventilator' in t]



total_df = pd.DataFrame.from_dict(updates)

In [18]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=total,
                    mode='lines',
                    name='Total Number of Cases'))

fig.update_layout(
    title_text='Total Number of Cases', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Infected', # yaxis label
)
fig.show()

with open('../plotly/ontario_total_cases.json', 'w') as f:
    f.write(fig.to_json())

# Doubling

In [19]:
doubling = [ (i+1)*math.log(2)/(math.log(p/positive[0])) for i,p in enumerate(positive[1:])]


In [20]:
fig = go.Figure(data=go.Scatter(x=dates[1:], y=doubling,
                    mode='lines',
                    name='Time to Double'))
fig.update_layout(title='Current Doubling Time',
                   xaxis_title='Date',
                   yaxis_title='Days to Double')
fig.show()
with open('../plotly/ontario_doubling.json', 'w') as f:
    f.write(fig.to_json())

# ACTIVE CASES

In [21]:
dead_percent = [d/t for d,t in zip(dead, total)]
recovered_percent = [r/t for r,t in zip(resolved, total)]
active_percent = [a/t for a,t in zip(positive, total)]

In [22]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x= dates, y=dead, name="Deceased"),

)

fig.add_trace(
    go.Scatter(x=dates, y=dead_percent, name="Deceased Percent", visible=False),
)


fig.add_trace(
    go.Scatter(x= dates, y=resolved, name="Recovered"),
    
)

fig.add_trace(
    go.Scatter(x=dates, y=recovered_percent, name="Recovered", visible=False),

)


fig.add_trace(
    go.Scatter(x= dates, y=positive, name="Active"),

)

fig.add_trace(
    go.Scatter(x=dates, y=active_percent, name="Active Percent", visible=False),
)

fig.update_layout(
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Number of Cases', # yaxis label

)

fig.update_layout(
    updatemenus=[
        dict(
            #type="buttons",
            direction="down",
            showactive = True,
            #active = 0,
            x=1,
            y=1.5,
            buttons=list([
                dict(label="Current Active",
                     method="update",
                     args=[{"visible": [True,False,True, False, True, False]},
                           {"title": 'Active, Deceased and Recovered Cases', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Number of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),
                dict(label="Percentage Active",
                     method="update",
                     args=[{"visible": [False,True,False, True, False, True]},
                           {"title": 'Percentage of Active, Deceased and Recovered Cases', 
                            "xaxis.title" : 'Date',
                            "yaxis.title" : 'Percentage of Cases',
                            "yaxis.type": "linear",
                            #"xaxis.type": "linear",
                            "annotations" : [],
                           }]   ),

            ]),
        )
    ])


fig.update_layout(
    title_text="Active Cases",
)
fig.update_layout(legend_orientation="h")
fig.update_layout(legend=dict( y=-0.4))


fig.show()

with open('../plotly/ontario_cases.json', 'w') as f:
    f.write(fig.to_json())

# Hospitalization


In [30]:
fig = go.Figure()

fig.add_trace(
    go.Scatter(x= dates[29:], y=hospitalization, name="Hospitalized"),

)

fig.add_trace(
    go.Scatter(x=dates[29:], y=icu, name="ICU"),
)


fig.add_trace(
    go.Scatter(x= dates[29:], y=ventilator, name="Ventilator"),
    
)

fig.add_trace(
    go.Scatter(x= dates[29:], y=[2000]*len(dates[29:]), name="Ventilator Capacity", mode = "lines"),
    
)

fig.update_layout(title='Hospitlization',
                   xaxis_title='Date',
                   yaxis_title='Number of Patients')

fig.show()

with open('../plotly/ontario_hospitilization.json', 'w') as f:
    f.write(fig.to_json())

# Age

In [24]:
final = updates[-1]

age_ranges = ['19 and under', '20-39', '40-59', '60-79', '80 and over']

ages_num = [final[a] for a in age_ranges]



In [25]:
fig = go.Figure([go.Bar(x=age_ranges, y=ages_num, marker_color = '#EB89B5')])

fig.update_layout(title='Cases per Age Group',
                   xaxis_title='Age Range',
                   yaxis_title='Number of Cases')

fig.show()

with open('../plotly/ontario_age.json', 'w') as f:
    f.write(fig.to_json())

# Testing

In [26]:

fig = go.Figure(data=[
    go.Bar(name='Negative', x=dates, y=negative),
    go.Bar(name='Under Investigation', x=dates, y=under),
    go.Bar(name='Confirmed Positive', x=dates, y=positive),
    go.Bar(name='Resolved', x=dates, y=resolved),
    go.Bar(name='Deceased', x=dates, y=dead),
])
fig.update_layout(
    title_text='Infection over time', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Infected', # yaxis label
    bargap=0.7, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)
# Change the bar mode
fig.update_layout(barmode='stack')
fig.show()

with open('../plotly/ontario_infection.json', 'w') as f:
    f.write(fig.to_json())

# Regional

In [10]:
city_infections = []
cities.remove("pending")
cities.remove("Unspecified")
dates = { t['date'][:10]:i for i, t in enumerate(all_cases)}

unique_dates_ix = list(dates.values())
dates = list(dates.keys())
dates = dates[3:]
dates.sort()
infections = [[case["date"][:10] for case in all_cases if case["city"]== city and case["date"][:10] > "2020-02-01"] for city in cities]

val = []
for infection in infections : 
    sorted_inf = sorted(infection)
    counter = collections.Counter(sorted_inf)
    for d in dates : 
        if d not in counter: 
            counter[d] = 0
    counter = {k: v for k, v in sorted(counter.items(), key=lambda item: item[0])}
    values = list(counter.values())
    val.append(values)
    city_infections.append( list(np.cumsum(values)))

In [11]:
color = ['#EB89B5', '#5b7bd6','#ff6960', '#d11411', '#00b159', '#00aedb', '#f37735', '#ffc425',
          '#a200ff', '#f47835', '#d41243', '#8ec127', '#feda75', '#fa7e1e', '#962fbf']*2


#fig = go.Figure()
fig = make_subplots(rows=8, cols=5, subplot_titles=tuple(cities), shared_xaxes=True, shared_yaxes = True)

for  i,c_i in enumerate(city_infections) :
    r = i//5+1
    c = i%5+1
    fig.add_trace(
    go.Scatter(x=dates, y=c_i),
    row=r, col=c
)

fig.update_layout(showlegend=False, title_text="Infection over time per City", height = 1000)
fig.show()

with open('../plotly/ontario_new_infections_by_city.json', 'w') as f:
    f.write(fig.to_json())

# Gender

In [12]:

df = pd.DataFrame.from_dict(all_cases)

male = [int(math.ceil(case["age"] / 10.0)) * 10 for case in all_cases if case["gender"] == "Male" and case["age"] != 'pending']
female = [case["age"] for case in all_cases if case["gender"] == "Female" and case["age"] != 'pending'] 


In [13]:


fig = go.Figure()
fig.add_trace(go.Histogram(x=male, marker_color = '#EB89B5'))


fig.update_layout(
    title_text='Male Infection by Age', # title of plot
    xaxis_title_text='Age ', # xaxis label
    yaxis_title_text='Infected', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

with open('../plotly/ontario_male_infection.json', 'w') as f:
    f.write(fig.to_json())

In [14]:

fig = go.Figure()
fig = go.Figure()
fig.add_trace(go.Histogram(x=female, marker_color = '#EB89B5'))


fig.update_layout(
    title_text='Female Infection by Age', # title of plot
    xaxis_title_text='Age', # xaxis label
    yaxis_title_text='Infected', # yaxis label
)

fig.show()

with open('../plotly/ontario_female_infection.json', 'w') as f:
    f.write(fig.to_json())

# Transmission Type

In [15]:


transmission = ['travel', 'close contact', 'pending']
infections = []
for t in transmission:
    temp = []
    for case in all_cases: 
        if "transmission" in case:
            if t in case['transmission']:
                temp.append(case["date"][:10])
    infections.append(temp)
            



In [16]:
import collections
cum_infections = []

for infection in infections[:2] : 
    sorted_inf = sorted(infection)
    counter = collections.Counter(sorted_inf)
    for d in dates : 
        if d not in counter: 
            counter[d] = 0
    counter = {k: v for k, v in sorted(counter.items(), key=lambda item: item[0])}
    values = list(counter.values())
    cum_infections.append( list(np.cumsum(values)))

In [17]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=dates, y=cum_infections[0],
                    mode='lines',
                    name='Travel'))
fig.add_trace(go.Scatter(x=dates, y=cum_infections[1],
                    mode='lines',
                    name='Close Contact'))
fig.update_layout(
    title_text='Cumulative Transmission over Time', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Number Infected', # yaxis label
)

fig.show()

with open('../plotly/ontario_transmission_type.json', 'w') as f:
    f.write(fig.to_json())

In [18]:
fig = go.Figure()
for  m, t in zip(infections, transmission) : 
    fig.add_trace(go.Histogram( x=m, name = t))

fig.update_layout(barmode='stack')

#fig = px.histogram(df, x="date", color = "city", color_discrete_sequence= px.colors.qualitative.Light24)
fig.update_layout(
    title_text='New transmission over time', # title of plot
    xaxis_title_text='Date', # xaxis label
    yaxis_title_text='Infected', # yaxis label
    bargap=0.2, # gap between bars of adjacent location coordinates
    bargroupgap=0.1 # gap between bars of the same location coordinates
)

fig.show()

In [44]:
with open("../data/ontario/total_ontario_2020-04-04T14:38:19.json", "r") as infile:
    ontario = json.load(infile)

In [45]:
ontario

{'Number of cases1': '3,630',
 'Change from previous report': '375',
 'Resolved2': '1,219',
 'Deceased3': '94',
 ' ': ' ',
 'Male': '1,755',
 'Female': '1,857',
 '19 and under': '93',
 '20-39': '1,025',
 '40-59': '1,319',
 '60-79': '903',
 '80 and over': '287',
 'Total Tested4': '71,338',
 'Currently Under Investigation5': '1,336',
 'Number of patients hospitalized with COVID-196': '506',
 'Number of patients in ICU7 with COVID-19': '196',
 'Number of patients in ICU7 on a ventilator with COVID-19': '152'}

In [46]:
{"date": "2020-04-02T17:39:33", "negative": 59056, "pending": 2052, "positive": 2793, "resolved": 831, "deceased": 53, "Total number of patients approved for COVID-19 testing to date": 62733}


{'date': '2020-04-02T17:39:33',
 'negative': 59056,
 'pending': 2052,
 'positive': 2793,
 'resolved': 831,
 'deceased': 53,
 'Total number of patients approved for COVID-19 testing to date': 62733}

In [48]:
with open("../data/ontario/all_updates.json", "r") as infile:
    updates = [json.loads(l) for l in infile]

In [50]:
for u in updates:
    u["total"] = u["positive"]
    u["positive"] = u["positive"] - u["resolved"] - u["deceased"]

In [71]:
with open("../data/ontario/updates.jsonl", "w") as outfile:
    for u in updates:
        json.dump(u,outfile)
        outfile.write('\n')

In [59]:
def map_keys(ontario):
    d = {}
    for key, value in ontario.items():
        if 'Number of cases' in key:
            d['total'] = int(value.replace(",","").replace("*",""))
        elif 'Change from previous report' in key :
            pass
        elif 'Resolved' in key : 
            d['resolved'] = int(value.replace(",","").replace("*",""))
        elif 'Deceased' in key:
            d['deceased'] = int(value.replace(",","").replace("*",""))
        elif 'Total Tested' in key : 
            d["total tested"] = int(value.replace(",","").replace("*",""))
        elif 'Currently Under Investigation' in key : 
            d['pending'] = int(value.replace(",","").replace("*",""))
            
        elif ' ' == key : 
            pass
        elif 'hospitalized' in key :
            d['hospitalizations'] = int(value.replace(",","").replace("*",""))
        elif 'ventilator' in key : 
            d["ventilator"] = int(value.replace(",","").replace("*",""))
        elif "ICU" in key : 
            d["ICU"] = int(value.replace(",","").replace("*",""))
        else :
            d[key] = int(value.replace(",","").replace("*",""))
        
            
    return d

In [64]:
new_update = map_keys(ontario)

In [61]:
data_directory = "../data/ontario/"
files = os.listdir(data_directory)
total_path = [ f for f in files if "total_ontario" in f][0]

In [62]:
total_path

'total_ontario_2020-04-04T14:38:19.json'

In [65]:
def get_date(file):
    date = file.split("_")[-1].split(".")[0]
    return date
date = get_date(total_path)

In [80]:
new_update["positive"] = new_update["total"] - new_update["resolved"] - new_update["deceased"]

In [81]:
new_update

{'total': 3630,
 'resolved': 1219,
 'deceased': 94,
 'Male': 1755,
 'Female': 1857,
 '19 and under': 93,
 '20-39': 1025,
 '40-59': 1319,
 '60-79': 903,
 '80 and over': 287,
 'total tested': 71338,
 'pending': 1336,
 'hospitalizations': 506,
 'ICU': 196,
 'ventilator': 152,
 'date': '2020-04-04T14:38:19',
 'positive': 2317}

In [147]:
with open("../data/ontario/updates.jsonl", 'a') as outfile:
    json.dump(new_update, outfile)
    outfile.write('\n')

In [141]:
new_update

{'total': 3630,
 'resolved': 1219,
 'deceased': 94,
 'Male': 1755,
 'Female': 1857,
 '19 and under': 93,
 '20-39': 1025,
 '40-59': 1319,
 '60-79': 903,
 '80 and over': 287,
 'total tested': 71338,
 'pending': 1336,
 'hospitalizations': 506,
 'ICU': 196,
 'ventilator': 152,
 'date': '2020-04-04T14:38:19',
 'positive': 2317}